In [11]:
import csv
import json
import mysql.connector
from mysql.connector import Error

# please modify hw4_config.json to your own config file
with open('hw4_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']


## 1-1: Input data into student table

In [12]:
try:
    # 連接 MySQL 資料庫
    conn = mysql.connector.connect(
        host=host,          
        user=user,        
        password=passwd)

    if conn.is_connected():
        print('is connected')
        cursor = conn.cursor(buffered=True)
        cursor.execute("CREATE DATABASE if not exists DB_class;")
        cursor.execute("use DB_class;")
        creating_table = '''create table if not exists student(
                                身份 char(30) not null,
                                系所 char(30) not null,
                                年級 int not null,
                                學號 char(30) not null primary key,
                                姓名 char(30) not null,
                                信箱 char(30) not null, 
                                班別 char(30) not null
                        )'''
        cursor.execute(creating_table)
        with open('DBMS_student_list.csv', encoding="utf-8-sig") as f:
            reader = csv.DictReader(f)
            for row in reader:
                inserting_row = "insert into DB_class.student values (%s, %s, %s, %s, %s, %s, %s);"
                cursor.execute(inserting_row, tuple(row.values()))
                conn.commit()
        print("successfully inserted data into database")
except Error as e:
    print(e)


is connected
successfully inserted data into database


## 1-2: Read the information about yourself

In [13]:
read_myself = "select * from DB_class.student where 姓名 like '謝佳穎%';"
cursor.execute(read_myself)
result = cursor.fetchone()
print(result)

('學生', '資管系', 2, 'R10725059', '謝佳穎 (HSIEH, CHIA-YING)', 'r10725059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-3: Update your own value of the "身份" column 

In [14]:
updating_myself = "update DB_class.student set 身份 = '特優生' where 姓名 like '謝佳穎%';"
cursor.execute(updating_myself)
conn.commit()
cursor.execute("select * from DB_class.student where 姓名 like '謝佳穎%';")
result = cursor.fetchone()
print(result)

('特優生', '資管系', 2, 'R10725059', '謝佳穎 (HSIEH, CHIA-YING)', 'r10725059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-4: Insert three new students 

In [15]:
with open('new_student_list.csv', encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        for row in reader:
                inserting_row = "insert into DB_class.student values (%s, %s, %s, %s, %s, %s, %s);"
                cursor.execute(inserting_row, tuple(row.values()))
                conn.commit()

cursor.execute('select * from DB_class.student')
result = cursor.fetchall()
for i in result:
    print(i)

cursor.close()

('旁聽生', '物理系', 4, 'B07202041', '林禹廷 (LIN, YU-TING)', 'b07202041@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '物理系', 4, 'B07202043', '溫進揚 (VOON JING YANG)', 'b07202043@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系', 4, 'B07303086', '李孟學 (lee meng-hsueh)', 'b07303086@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '生傳系', 4, 'B07610058', '盧家雯 (LU,CHIA-WEN)', 'b07610058@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('旁聽生', '經濟系', 4, 'B08303013', '沈汶鋒 (SHEN , WUN-FONG)', 'b08303013@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系', 4, 'B08303047', '黃翔易 (HUANG,XIANG-YI)', 'b08303047@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '經濟系', 4, 'B08303098', '張暐翔 (CHANG,WEI-SIANG)', 'b08303098@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '心理系', 4, 'B08305037', '周彥成 (CHOU YAN CHENG)', 'b08305037@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '醫工系', 4, 'B08508002', '郭思言 (KUO,SZU-YEN)', 'b08508002@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '生物機電系', 4, 'B08611048', '賴鴻昇 (LAI

## 1-5: Prepared select statement (read back the information of any student given his/her ID)

In [16]:
cursor = conn.cursor(buffered=True, prepared=True)

selected_ids = ['R10123456', 'B09987653', 'R11123001']

for i in selected_ids:
    checking_info = "select * from DB_class.student where 學號 = %s"
    cursor.execute(checking_info, [i])
    result = cursor.fetchone()
    print(result)
    

('旁聽生', '電機所', 2, 'R10123456', '小紅', 'r10123456@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('學生', '物理系', 3, 'B09987653', '小黃', 'b09987653@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')
('觀察者', '電信所', 1, 'R11123001', '小綠', 'r11123001@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)')


## 1-6: Group information

In [17]:
altering_table1 = "alter table student add final_group int"
altering_table2 = "alter table student add final_captain char(20)"
cursor.execute(altering_table1)
conn.commit()
cursor.execute(altering_table2)
conn.commit()

cursor.execute("describe `student`;")
result = cursor.fetchall()
for i in result:
    print(i)

('身份', 'char(30)', 'NO', '', None, '')
('系所', 'char(30)', 'NO', '', None, '')
('年級', 'int', 'NO', '', None, '')
('學號', 'char(30)', 'NO', 'PRI', None, '')
('姓名', 'char(30)', 'NO', '', None, '')
('信箱', 'char(30)', 'NO', '', None, '')
('班別', 'char(30)', 'NO', '', None, '')
('final_group', 'int', 'YES', '', None, '')
('final_captain', 'char(20)', 'YES', '', None, '')


In [18]:
cursor.execute("select count(*) from DB_class.student;")
total_row_count = cursor.fetchall()

try:
    for i in total_row_count:
        updating_group = '''update DB_class.student 
                        set final_group = case when 學號 in ('R10725059', 'R11725058', 'R11725047', 'R11725044') then 12 else 0 end'''
        cursor.execute(updating_group)
        updating_captain = '''update DB_class.student 
                        set final_captain = case when 學號 in ('R11725044') then "Y" else "N" end'''
        cursor.execute(updating_captain)
        conn.commit()

    print("successfully updated group member info.")

except Error as e:
    print(e)

successfully updated group member info.


In [19]:
cursor.execute("select * from student where final_group = 12")
result = cursor.fetchall()
for i in result:
    print(i)

('特優生', '資管系', 2, 'R10725059', '謝佳穎 (HSIEH, CHIA-YING)', 'r10725059@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 12, 'N')
('學生', '資管系', 1, 'R11725044', '鄧鈺儒 (TENG, YU-JU)', 'r11725044@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 12, 'Y')
('學生', '資管系', 1, 'R11725047', '林品歷 (LIN, PIN-LI)', 'r11725047@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 12, 'N')
('學生', '資管系', 1, 'R11725058', '吳品萱 (\x08PIN-HSUAN WU)', 'r11725058@ntu.edu.tw', '資料庫系統-從SQL到NoSQL (EE5178)', 12, 'N')


In [20]:
cursor.close()
conn.close()